In [1]:
import os 
packages = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages {0} pyspark-shell".format(packages)
)

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [2]:
df1 = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "kafka:9092")\
.option("subscribe", "Trump")\
.load()

In [3]:
query2 = df1.writeStream\
    .queryName("device_counts").format("memory")\
    .start()

In [20]:
spark.sql('select count(*) FROM device_counts').show()
df2 = spark.sql('SELECT * FROM device_counts').toPandas()

Py4JJavaError: An error occurred while calling o71.showString.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.catalyst.expressions.UnsafeRow.copy(UnsafeRow.java:489)
	at org.apache.spark.sql.catalyst.expressions.UnsafeRow.copy(UnsafeRow.java:62)
	at org.apache.spark.sql.execution.SparkStrategies$BasicOperators$.$anonfun$apply$31(SparkStrategies.scala:687)
	at org.apache.spark.sql.execution.SparkStrategies$BasicOperators$$$Lambda$2565/0x00000008410dc040.apply(Unknown Source)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.TraversableLike$$Lambda$60/0x000000084015c040.apply(Unknown Source)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLik

In [18]:
# df2['value'] = df2['value'].apply(lambda x: x.decode('utf-8'))
df2.size

41475

In [19]:
df1.writeStream.trigger(processingTime='5 seconds')\
.format("console").outputMode("complete").start()

AnalysisException: Complete output mode not supported when there are no streaming aggregations on streaming DataFrames/Datasets;;
kafka